In [ ]:
import pandas as pd
import plotly.graph_objects as go
import re

# === 1. Daten einlesen und vorbereiten ===

# Datei einlesen
df = pd.read_excel("01.01.2015-31.12.2024_Sonnenscheindaten_001.xlsx", sheet_name="Sheet1")

# Koordinaten in Dezimalgrad umrechnen
def convert_to_decimal(coord):
    match = re.match(r"(\d+)°(\d+)'", coord)
    if match:
        degrees = int(match.group(1))
        minutes = int(match.group(2))
        return degrees + minutes / 60
    return None

df["Longitude_decimal"] = df["Longitude"].apply(convert_to_decimal)
df["Latitude_decimal"] = df["Latitude"].apply(convert_to_decimal)

# Gruppieren und kumulieren
grouped = df.groupby(["Jahr", "Ort_x", "Latitude_decimal", "Longitude_decimal"]).agg({"Sonnenstunden": "sum"}).reset_index()
grouped["Sonnenstunden_kumuliert"] = grouped.groupby("Ort_x")["Sonnenstunden"].cumsum()
grouped["Sonnenstunden_kumuliert_real"] = grouped["Sonnenstunden_kumuliert"]  # Realwerte

jahre = sorted(grouped["Jahr"].unique())

# === 2. Farb- und Linienbreitenfunktionen ===

def get_color(value):
    if value <= 2500: return "rgb(0, 0, 139)"       # Dunkelblau
    elif value <= 5000: return "rgb(30, 144, 255)"  # Blau
    elif value <= 7500: return "rgb(60, 179, 113)"  # Grün
    elif value <= 10000: return "rgb(124, 252, 0)"  # Hellgrün
    elif value <= 12500: return "rgb(255, 255, 0)"  # Gelb
    elif value <= 15000: return "rgb(255, 165, 0)"  # Orange
    elif value <= 17500: return "rgb(255, 69, 0)"   # Rot
    elif value <= 20000: return "rgb(220, 20, 60)"  # Dunkelrot
    else: return "rgb(139, 0, 0)"                   # Sehr dunkelrot

def get_line_width(value):
    if value < 5000: return 2
    elif value < 10000: return 3
    elif value < 15000: return 4
    elif value < 20000: return 5
    else: return 6

# === 3. Start-Frame ===

df_start = grouped[grouped["Jahr"] == jahre[0]]
start_traces = [
    go.Scatter3d(
        x=[row["Longitude_decimal"]]*2,
        y=[row["Latitude_decimal"]]*2,
        z=[0, row["Sonnenstunden_kumuliert_real"]],
        mode="lines",
        line=dict(
            color=get_color(row["Sonnenstunden_kumuliert"]),
            width=get_line_width(row["Sonnenstunden_kumuliert"])
        ),
        text=f"Ort: {row.Ort_x}<br>Jahr: {jahre[0]}<br>Sonnenstunden: {int(row.Sonnenstunden_kumuliert):,}",
        hoverinfo="text",
        showlegend=False
    ) for _, row in df_start.iterrows()
]

# === 4. Animations-Frames ===

frames = []
for jahr in jahre:
    df_jahr = grouped[grouped["Jahr"] == jahr]
    traces = [
        go.Scatter3d(
            x=[row["Longitude_decimal"]]*2,
            y=[row["Latitude_decimal"]]*2,
            z=[0, row["Sonnenstunden_kumuliert_real"]],
            mode="lines",
            line=dict(
                color=get_color(row["Sonnenstunden_kumuliert"]),
                width=get_line_width(row["Sonnenstunden_kumuliert"])
            ),
            text=f"Ort: {row.Ort_x}<br>Jahr: {jahr}<br>Sonnenstunden: {int(row.Sonnenstunden_kumuliert):,}",
            hoverinfo="text",
            showlegend=False
        ) for _, row in df_jahr.iterrows()
    ]
    frames.append(go.Frame(data=traces, name=str(jahr)))

# === 5. Layout ===

layout = go.Layout(
    title="☀️ Kumulierte Sonnenstunden in der Schweiz (2015–2024)",
    scene=dict(
        xaxis=dict(title="Längengrad", showgrid=True, zeroline=False, showbackground=False),
        yaxis=dict(title="Breitengrad", showgrid=True, zeroline=False, showbackground=False),
        zaxis=dict(
            title="Kumulierte Sonnenstunden",
            tickvals=[5000, 10000, 15000, 20000],
            ticktext=["5'000", "10'000", "15'000", "20'000"],
            range=[0, grouped["Sonnenstunden_kumuliert_real"].max()],
            showgrid=True
        ),
        camera=dict(eye=dict(x=2.0, y=1.5, z=0.8)),
        aspectmode='manual',
        aspectratio=dict(x=2, y=1.5, z=0.6)
    ),
    sliders=[{
        "steps": [{
            "method": "animate",
            "label": str(j),
            "args": [[str(j)], {"mode": "immediate", "frame": {"duration": 600, "redraw": True}}]
        } for j in jahre]
    }],
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None]},
            {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0}}]}
        ]
    }]
)

# === 6. Plot erzeugen & speichern ===

fig = go.Figure(data=start_traces, layout=layout, frames=frames)
fig.write_html("sonnenstunden_3d_dynamische_breite.html")


FileNotFoundError: [Errno 2] No such file or directory: '01.01.2015-31.12.2024_Sonnenscheindaten_001.xlsx'